In [3]:
import sys; sys.path.append('../3rdparty/elastic_rods/python'); sys.path.append('..'); sys.path.append('../../python')
import numpy as np, elastic_rods, linkage_vis
from bending_validation import suppress_stdout as so
from linkage_vis import LinkageViewer
from elastic_rods import InterleavingType
import elasticRodsGH as gh


ModuleNotFoundError: No module named 'linkage_vis'

#### XShell Initialization
From Json file generated with GH

In [2]:
GHfile = "Test_xshell.json"

# Import json from GH 
data = gh.linkageData(GHfile)
linkage = data.linkage

# Initialize viewer
view = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
view.setCameraParams(((2.3969, -1.1451, -0.5262), (0.0765, 0.0848, -0.9935), (0.1111, 0.1865, 0.5316)))
view.averagedMaterialFrames = True
view.averagedCrossSections = True
view.show()

/Users/suzuki/opt/miniconda3/envs/cshells/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), …

#### Step 1: Compute initial equilibrium

In [9]:
# Joint indexes can be checked with GH
# Support conditions can be set with GH. If no support conditions are provided, the central joint is taken.
supports = data.supports

# Run equilibrium
with so(): elastic_rods.compute_equilibrium(linkage, fixedVars=supports)

# Update viewer    
view.update()
 

#### Step 2: Compute Deployment

In [10]:
   
from open_linkage import open_linkage

# Custom method for setting Newton's solver options (Optional)
def equilibriumSolver(tgtAngle, linkage, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(linkage, tgtAngle, options=opts, fixedVars=fv)

driver = linkage.centralJoint()
target_angle = linkage.averageJointAngle + np.deg2rad(15)
with so(): open_linkage(linkage, driver, target_angle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True)